In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk
import random
import shapefile
import json
import math

import geopy.distance

from tqdm import tqdm

from multiprocessing import Pool
from IPython.display import clear_output

In [2]:
def read_shapefile(sf_shape):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """

    fields = [x[0] for x in sf_shape.fields][1:]
    records = [y[:] for y in sf_shape.records()]
    #records = sf_shape.records()
    shps = [s.points for s in sf_shape.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df


In [3]:
#загружаем информацию о сеточном разбиении МО в память
myshp = open("fishnet2021.shp", "rb")
mydbf = open("fishnet2021.dbf", "rb")
r = shapefile.Reader(shp=myshp, dbf=mydbf)
s_df = read_shapefile(r)
#в поле coords первая координата центр квадрата, остальные его углы
s_df.head()

,cell_zid,coords
0,0,"[(38.64720691553791, 54.255243332408604), (38...."
1,1,"[(38.65487820205851, 54.255170084754035), (38...."
2,2,"[(38.66254945426432, 54.255096350709195), (38...."
3,3,"[(38.670220672771414, 54.25502212698049), (38...."
4,4,"[(38.67789185552521, 54.25494741523556), (38.6..."


In [4]:
#считываем информацию о переходах Дом-Работа и прикручиваем туда координаты переходов
h_w_matrix = pd.read_csv("04_CMatrix_Home_Work_July.csv")
h_w_matrix = h_w_matrix.merge(right=s_df, how='inner', left_on='home_zid', right_on='cell_zid')
h_w_matrix = h_w_matrix.merge(right=s_df, how='inner', left_on='work_zid', right_on='cell_zid')
h_w_matrix.drop('cell_zid_x', axis=1, inplace=True)
h_w_matrix.drop('cell_zid_y', axis=1, inplace=True)

h_w_matrix.head()

,home_zid,work_zid,customers_cnt,coords_x,coords_y
0,33617,35383,3,"[(37.04909381713915, 55.14932918598869), (37.0...","[(37.13516508529589, 55.172076772159485), (37...."
1,33620,33621,2,"[(37.07262354156165, 55.149413945205254), (37....","[(37.08046681019148, 55.14944118799408), (37.0..."
2,33621,33621,2,"[(37.08046681019148, 55.14944118799408), (37.0...","[(37.08046681019148, 55.14944118799408), (37.0..."
3,33622,38813,1,"[(37.08831009005705, 55.14946792727634), (37.0...","[(37.01696105787737, 55.21209316621927), (37.0..."
4,38412,38813,1,"[(37.04057941677159, 55.207692480896384), (37....","[(37.01696105787737, 55.21209316621927), (37.0..."


In [5]:
#считываем наименование всех регинов и прикручиваем к ним их координаты
#loc_names = pd.read_excel('names.xlsx')
#loc_names['cell_zid'] = loc_names['cell_zid'].map(int)
#loc_names['adm_zid'] = loc_names['adm_zid'].map(int)
#loc_names = loc_names.merge(right=s_df, how='inner', left_on='cell_zid', right_on='cell_zid')
loc_names = pd.read_csv('rebuilded_names.csv')
loc_names.head()

,Unnamed: 0,cell_zid,area_peresechenia_s_admzone_kv.km,adm_zid,adm_name,okrug_name,sub_ter
0,0,32909,0.407116,216,Роговское,Троицкий административный округ,Новая Москва
1,1,32910,1.003458,216,Роговское,Троицкий административный округ,Новая Москва
2,2,33261,0.233312,216,Роговское,Троицкий административный округ,Новая Москва
3,3,33262,1.868032,216,Роговское,Троицкий административный округ,Новая Москва
4,4,33263,2.500630,216,Роговское,Троицкий административный округ,Новая Москва


In [6]:
#считываем информацию о плотности проживающего, работающего и проходящего населения для каждого квадрата и прикручиваем туда координаты квадратов
c_locations = pd.read_csv("01_CLocation_July.csv")
c_locations = c_locations.merge(right=s_df, how='inner', left_on='zid', right_on='cell_zid')
c_locations.drop('cell_zid', axis=1, inplace=True)
#c_locations['coords'] = c_locations['coords'].apply(lambda x: (x[0][1],x[0][0] )) # оставляем только 1 координату
c_locations['lat'] = c_locations['coords'].map(lambda x: x[0][1]) # извлекаем широту
c_locations['lon'] = c_locations['coords'].map(lambda x: x[0][0]) # извлекаем долготу

c_locations.drop('coords', axis=1, inplace=True)


c_locations.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,4,0,0,0,55.140287,37.033512
1,32910,25,8,26,0,55.140316,37.041354
2,33261,2,0,2,0,55.144748,37.025618
3,33262,13,2,11,0,55.144778,37.033460
4,33263,10,0,3,0,55.144808,37.041302


In [7]:
adm_names = loc_names['adm_name'].drop_duplicates()
print("Кол-во уникальных райнов: ",adm_names.shape[0])

Кол-во уникальных райнов:  146


In [8]:
c_locations.loc[c_locations['zid'].isin(loc_names.loc[loc_names['adm_name'] == 'Северное Тушино']['cell_zid'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 9463 to 9764
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zid                 50 non-null     int64  
 1   customers_cnt_home  50 non-null     int64  
 2   customers_cnt_job   50 non-null     int64  
 3   customers_cnt_day   50 non-null     int64  
 4   customers_cnt_move  50 non-null     int64  
 5   lat                 50 non-null     float64
 6   lon                 50 non-null     float64
dtypes: float64(2), int64(5)
memory usage: 3.1 KB


Сгенерируем псевдоданные за предыдущий месяц:

In [9]:
#Изменяем исходный датасет июля на рандомные % чтобы сгененрировать динамику плотности населения
c_locations_june = c_locations.copy()
c_locations_june['customers_cnt_home'] = c_locations_june['customers_cnt_home'].apply(lambda x: int(float(x) *( random.uniform(0.5, 1.7))))
c_locations_june['customers_cnt_job'] = c_locations_june['customers_cnt_job'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))
c_locations_june['customers_cnt_day'] = c_locations_june['customers_cnt_day'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))
c_locations_june['customers_cnt_move'] = c_locations_june['customers_cnt_move'].apply(lambda x: int(float(x) *( random.uniform(0.8, 1.1))))

c_locations_june.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon
0,32909,4,0,0,0,55.140287,37.033512
1,32910,37,8,24,0,55.140316,37.041354
2,33261,3,0,1,0,55.144748,37.025618
3,33262,21,1,9,0,55.144778,37.033460
4,33263,12,0,3,0,55.144808,37.041302


In [10]:
delta_cnt_df = c_locations_june.copy()
delta_cnt_df = delta_cnt_df.merge(right=c_locations, how='inner', left_on='zid', right_on='zid')
delta_cnt_df.drop(['lat_y', 'lon_y'], axis=1, inplace=True)
delta_cnt_df.rename({'lat_x':'lat', 'lon_x':'lon'}, axis=1, inplace=True)

delta_cnt_df.head()

,zid,customers_cnt_home_x,customers_cnt_job_x,customers_cnt_day_x,customers_cnt_move_x,lat,lon,customers_cnt_home_y,customers_cnt_job_y,customers_cnt_day_y,customers_cnt_move_y
0,32909,4,0,0,0,55.140287,37.033512,4,0,0,0
1,32910,37,8,24,0,55.140316,37.041354,25,8,26,0
2,33261,3,0,1,0,55.144748,37.025618,2,0,2,0
3,33262,21,1,9,0,55.144778,37.033460,13,2,11,0
4,33263,12,0,3,0,55.144808,37.041302,10,0,3,0


In [11]:
#расчёт жельты изменения плотностей
for feature_type in ['home','job','day', 'move']:
    delta_cnt_df[f'customers_cnt_{feature_type}_y'] = delta_cnt_df[f'customers_cnt_{feature_type}_x'] - delta_cnt_df[f'customers_cnt_{feature_type}_y']
    delta_cnt_df.rename({f'customers_cnt_{feature_type}_y':f'customers_dlt_{feature_type}'},axis=1, inplace=True)
    delta_cnt_df.rename({f'customers_cnt_{feature_type}_x':f'customers_cnt_{feature_type}'},axis=1, inplace=True)

delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move
0,32909,4,0,0,0,55.140287,37.033512,0,0,0,0
1,32910,37,8,24,0,55.140316,37.041354,12,0,-2,0
2,33261,3,0,1,0,55.144748,37.025618,1,0,-1,0
3,33262,21,1,9,0,55.144778,37.033460,8,-1,-2,0
4,33263,12,0,3,0,55.144808,37.041302,2,0,0,0


In [13]:
tqdm.pandas(desc="find logistic!")
delta_cnt_df['logistic'] = 0
delta_cnt_df['logistic'] = delta_cnt_df['zid'].progress_apply(lambda x: sum(h_w_matrix.loc[h_w_matrix['work_zid'] == x]['customers_cnt'].values) + sum(h_w_matrix.loc[h_w_matrix['home_zid'] == x]['customers_cnt'].values))


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find logistic!: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 10240/10240 [00:25<00:00, 407.64it/s]


In [14]:
delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move,logistic
0,32909,4,0,0,0,55.140287,37.033512,0,0,0,0,0
1,32910,37,8,24,0,55.140316,37.041354,12,0,-2,0,0
2,33261,3,0,1,0,55.144748,37.025618,1,0,-1,0,0
3,33262,21,1,9,0,55.144778,37.033460,8,-1,-2,0,2
4,33263,12,0,3,0,55.144808,37.041302,2,0,0,0,0


In [15]:
mfc_df = pd.read_excel('mos_coords.xlsx')
mfc_df['geodata_center'] = mfc_df['geodata_center'].apply(lambda x: (json.loads(x)['coordinates'][1],json.loads(x)['coordinates'][0])  )
mfc_df['lon'] = mfc_df['geodata_center'].apply(lambda x: x[0])
mfc_df['lat'] = mfc_df['geodata_center'].apply(lambda x: x[1])
#mfc_df['ShortName', 'WindowCount', 'lon', 'lat', 'District'].to_csv('mos_coords.csv')

mfc_new = pd.DataFrame(columns=['ShortName', 'WindowCount', 'lon', 'lat', 'District'])

for name in ['ShortName', 'WindowCount', 'lon', 'lat', 'District', 'geodata_center']:
    mfc_new[name] = mfc_df[name]

mfc_new.to_csv("mos_coords.csv")
mfc_df=mfc_new
mfc_df.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ShortName       139 non-null    object 
 1   WindowCount     139 non-null    int64  
 2   lon             139 non-null    float64
 3   lat             139 non-null    float64
 4   District        139 non-null    object 
 5   geodata_center  139 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 6.6+ KB


In [16]:
tqdm.pandas(desc="find mfc distance!")


delta_cnt_df['nearest_mfc_distance'] = 0
delta_cnt_df['nearest_mfc_distance'] = delta_cnt_df['zid'].progress_apply(
    lambda x: geopy.distance.distance(mfc_df.loc[mfc_df['geodata_center'].apply(
        lambda y: geopy.distance.distance((y[0],y[1]),
                          (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],
                           delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])
                                    ).m
    ).idxmin()]['geodata_center'], (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],
                           delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])).m)
    
delta_cnt_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find mfc distance!: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 10240/10240 [32:20<00:00,  5.28it/s]
C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find mfc windows!:   0%|                                                                                                 | 1/10240 [00:00<31:52,  5.35it/s]


KeyError: 'Window'

In [17]:

tqdm.pandas(desc="find mfc windows!")
delta_cnt_df['nearest_mfc_windows'] = delta_cnt_df['zid'].progress_apply(
    lambda x: mfc_df.loc[mfc_df['geodata_center'].apply(
        lambda y: geopy.distance.distance((y[0],y[1]),
                          (delta_cnt_df.loc[delta_cnt_df['zid']==x]['lat'].values[0],
                           delta_cnt_df.loc[delta_cnt_df['zid']==x]['lon'].values[0])
                                    ).m
    ).idxmin()]['WindowCount'])
    
delta_cnt_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
find mfc windows!: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10240/10240 [32:22<00:00,  5.27it/s]


,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move,logistic,nearest_mfc_distance,nearest_mfc_windows
0,32909,4,0,0,0,55.140287,37.033512,0,0,0,0,0,43617.483646,47
1,32910,37,8,24,0,55.140316,37.041354,12,0,-2,0,0,43407.995189,47
2,33261,3,0,1,0,55.144748,37.025618,1,0,-1,0,0,43381.461499,47
3,33262,21,1,9,0,55.144778,37.033460,8,-1,-2,0,2,43165.034942,47
4,33263,12,0,3,0,55.144808,37.041302,2,0,0,0,0,42953.339821,47


In [148]:
print('max:', delta_cnt_df['nearest_mfc'].max(), ' min:', delta_cnt_df['nearest_mfc'].min() )

max: 43617.48364582916  min: 20.809329400719918


In [190]:
delta_cnt_df['mfc_chance'] = 0

#Настройка влияния весов на параметры плотности людей
alphas = {'home':1.0,'job':1.0,'day':1.0, 'move':1.0}
alphas_dlt = {'home':0.5,'job':0.5,'day':0.5, 'move':0.5}


for feature in ['home', 'job', 'day', 'move']:
    delta_cnt_df['mfc_chance'] = delta_cnt_df['mfc_chance'] + alphas[feature] * delta_cnt_df[f'customers_cnt_{feature}']
for feature in ['home', 'job', 'day', 'move']:
    delta_cnt_df['mfc_chance'] = delta_cnt_df['mfc_chance'] + alphas_dlt[feature] *  delta_cnt_df[f'customers_dlt_{feature}']  
    
delta_cnt_df['mfc_chance'] = delta_cnt_df['mfc_chance'] + (delta_cnt_df['nearest_mfc'])# / 43617.48364582916)*1000
delta_cnt_df['mfc_chance'] = delta_cnt_df['mfc_chance'] + (delta_cnt_df['logistic'])

delta_cnt_df['mfc_chance'] = delta_cnt_df['mfc_chance'].progress_apply(lambda x: 1 + 10* x / 42070.344117)
    
    #.progress_apply()# = delta_cnt_df['customers_cnt_home']*alphas['home'] + 

find mfc progress!: 100%|████████████████████████████████████████████████████████████████████████████████████████| 10240/10240 [00:00<00:00, 852835.98it/s]


In [155]:
 print(delta_cnt_df.loc[ delta_cnt_df['nearest_mfc'].idxmin()]['lat'], delta_cnt_df.loc[ delta_cnt_df['nearest_mfc'].idxmin()]['lon'])

55.72936609379239 37.734896667645025


In [191]:
delta_cnt_df.to_csv('june_full_data.csv')
delta_cnt_df.head()

,zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,lat,lon,customers_dlt_home,customers_dlt_job,customers_dlt_day,customers_dlt_move,nearest_mfc,logistic,mfc_chance
0,32909,2,0,0,0,55.140287,37.033512,-2,0,0,0,43617.483646,0,11.367988
1,32910,13,6,26,0,55.140316,37.041354,-12,-2,0,0,43407.995189,0,11.326988
2,33261,2,0,2,0,55.144748,37.025618,0,0,0,0,43381.461499,0,11.312600
3,33262,13,1,11,0,55.144778,37.033460,0,-1,0,0,43165.034942,2,11.266504
4,33263,7,0,2,0,55.144808,37.041302,-3,0,-1,0,42953.339821,0,11.211549


In [193]:
print(delta_cnt_df.loc[ delta_cnt_df['mfc_chance'].idxmax()])

zid                   32909.000000
customers_cnt_home        2.000000
customers_cnt_job         0.000000
customers_cnt_day         0.000000
customers_cnt_move        0.000000
lat                      55.140287
lon                      37.033512
customers_dlt_home       -2.000000
customers_dlt_job         0.000000
customers_dlt_day         0.000000
customers_dlt_move        0.000000
nearest_mfc           43617.483646
logistic                  0.000000
mfc_chance               11.367988
Name: 0, dtype: float64
